In [2]:
pip install qiskit

  Using cached rustworkx-0.17.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached scipy-1.16.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (61 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 2.9 MB/s  0:00:02 eta 0:00:01
Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached rustworkx-0.17.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
Using cached scipy-1.16.1-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (35.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [qiskit]2m6/7 [qiskit]kx]
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install qiskit_ibm_runtime

  Using cached requests_ntlm-1.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached ibm_cloud_sdk_core-3.24.2-py3-none-any.whl.metadata (8.7 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pyspnego-0.11.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached cffi-1.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.2 MB/s  0:00:00 eta 0:00:01
Using cached ibm_cloud

In [4]:
pip install qiskit_aer

  Using cached qiskit_aer-0.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.3 kB)
Using cached qiskit_aer-0.17.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Note: you may need to restart the kernel to use updated packages.


Modules:
  - QuantumCircuit
  - AerSimulator
  - backend.run
  - transpile (optional)
  - random number generator (Python or Qiskit tools)

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit_aer import AerSimulator, AerProvider
from qiskit_ibm_runtime.fake_provider import FakeKyoto
import random

''' Noise Model '''
import numpy as np #For random numbers
from qiskit_aer.noise import NoiseModel, depolarizing_error

''' Hashing library '''
from hashlib import sha256

''' For testing '''
import unittest

#High Level Pseusocode
	1.	Alice generates random bits + bases
	2.	Bob generates random bases
	3.	Alice encodes bits using her bases
	4.	Bob measures using his bases
	5.	Compare bases over classical channel
	6.	Discard mismatched basis results
	7.	Use matching bits as secret key


In [ ]:
'''
   Args:
    n (int) : A length of bitstring

  Return:
    (str) : A random bitsting length n
'''

def random_bits_string(n):
  return ''.join(random.choice('01') for _ in range(n))

In [ ]:
'''
  Args:
    n (int) : A length of sequence of basis

  Return:
    (str) : A random sequence of basis length n
'''

def random_basis(n):
  return ''.join(random.choice('ZX') for _ in range(n))

In [ ]:
'''
  Args:
    bits (str) : Alice's bitstring which to be encoded

  Return:
    (QuantumCircuit) : Encoded qubit
'''

def encode_qubits(bits):
  qreg = QuantumRegister(len(bits))
  creg = ClassicalRegister(len(bits))
  circuit = QuantumCircuit(qreg, creg)

  # enumerate() does manage index number and its value pair all them once in input data structure
  # index i and data bit in bits
  
  for i,bit in enumerate(bits):
    if bit == '1':
      circuit.x(qreg[i])
  print(circuit.to_instruction())
  return circuit

In [ ]:
'''
  Args:
    circuit (QuantumCircuit) : The qubit would be measured by Bob
    bob_basis (str) : The bases Bob use to measure the qubit from Alice

  Return:
    (QuantumCircuit) : The circuit which is added measurement devices
'''

def measure_qubits(circuit, bob_basis):
  for i in range(len(bob_basis)):
    if bob_basis[i] == 'X':
      circuit.h(i)

  circuit.measure_all(inplace=True)

  return circuit

In [ ]:
'''
 Args:
    circuit (QuantumCircuit) : The qubit would be transpiled
    bob_bits (list) : An empty list, would stroe measurement data

  Return:
    (list) : list of measurement data regarded its counts
'''

def run_simulator(circuit, bob_bits, shot=1024):
  
  # For invoking fake provider
  #backend = FakeKyoto()
  #simulator = AerSimulator.from_backend(backend) #initialize aer-simulator from backend
  
  # Normal AerSimulator
  simulator = AerSimulator()
  transpiled = transpile(circuit, simulator)
  result = simulator.run(transpiled, memory=True, shot=shot).result()

  # Obtain measurement data into bob_bits
  memory = result.get_memory(transpiled)
  bob_bits.append(memory)

  return result.get_counts(transpiled)

In [ ]:
'''
  Args:
    circuit (QuantumCircuit) : The qubit would be transpiled
    bob_bits (list) : An empty list, would stroe measurement data

  Return:
    (list) : list of measurement data regarded its counts
'''

def run_simulator_with_noise(circuit,bob_bits,shot=1024):
  # Create NoiseModel
  error = depolarizing_error(0.5,1)
  noise_model = NoiseModel()
  noise_model.add_all_qubit_quantum_error(error,'x')

  # AerSimulator with noise
  simulator = AerSimulator(noise_model=noise_model)
  transpiled = transpile(circuit, simulator)
  result = simulator.run(transpiled, memory=True,shot=shot).result()

  # Obtain measurement data into bob_bits
  memory = result.get_memory(transpiled)
  bob_bits.append(memory)

  return result.get_counts(transpiled)


In [ ]:
'''
Args:
    alice_basis (str) : Alice's basis
    bob_basis (str) : Bob's basis
    alice_bits (str) : Initial Alice's bits

  Return:
    (str) : The shared secret key
'''

def sift_keys(alice_basis, bob_basis, alice_bits):
  shared_key = ''
  for i in range(len(alice_basis)):
    if alice_basis[i] == bob_basis[i]:
      shared_key += alice_bits[i]
  return shared_key

In [ ]:
'''
  Calculates the Quantum Bit Error Rate (QBER).

  Args:
    alice_bits (str) : Alice's generated bit string.
    bob_measurements (list) : A list of measurement outcomes from Bob's side
                             (each element is a string representing a shot).
    alice_bases (str) : Alice's chosen bases string.
    bob_bases (str) : Bob's chosen bases string.

  Returns:
    (float) : The calculated QBER.
'''

def qber(alice_bits, bob_measurements, alice_basis, bob_basis):
  matching_basis_bits_count = 0
  mismatched_bits_count = 0

  # Iterate through each shot
  for shot_measurement in bob_measurements:
    # Iterate through each qubit in the shot
    for i in range(len(alice_bits)):
      # Check if bases match for this qubit
      if alice_basis[i] == bob_basis[i]:
        matching_basis_bits_count += 1
        bob_measured_bit = shot_measurement[i]
        # Compare Alice's bit with Bob's measured bit for this qubit in this shot
        if alice_bits[i] != bob_measured_bit[i]:
          mismatched_bits_count += 1

  if matching_basis_bits_count == 0:
    # Avoid division by zero if no bases match
    return 0.0
  print(f"Mismatched bits count: {mismatched_bits_count}")
  print(f"Matching bases bits count: {matching_basis_bits_count}")

  return mismatched_bits_count / matching_basis_bits_count

In [ ]:
'''
  Return:
    key (str) : The probably secure key
'''
def bb84():
  n = 29
  alice_bits = random_bits_string(n)
  alice_basis = random_basis(n)
  bob_basis = random_basis(n)

  circuit = encode_qubits(alice_bits)
  bob_measurement = measure_qubits(circuit, bob_basis)
  bob_bits = []
  #results = run_simulator(bob_measurement, bob_bits) # without NoiseModel
  results = run_simulator_with_noise(bob_measurement,bob_bits) # with NoiseModel

  # Obtaining the secure secret key
  key = sift_keys(alice_basis, bob_basis, alice_bits)

  # Transpilation(optional)
  transpiled = transpile(bob_measurement, AerSimulator())

  # Result
  print("QBER: " + str(qber(alice_bits, bob_bits, alice_basis, bob_basis)))

  return key if qber(alice_bits, bob_bits, alice_basis, bob_basis) < 0.2 else "None"

In [ ]:
'''
  Arg:
    key (str) : The secret key
  Return:
    (str) : The encoded key by sha256 hash function
'''

def amplification(key):
  return sha256(bytes(key, 'utf-8')).hexdigest()

In [ ]:

if __name__ == '__main__':
  key = bb84()
  h = amplification(key) if key != 'None' else 'Non-secure key'
  print("This is the key: " + str(key))
  print("This is after amplification: " + h) 

Round 1
Instruction(name='circuit-171', num_qubits=29, num_clbits=29, params=[])
Mismatched bits count: 8
Matching bases bits count: 15
QBER: 0.5333333333333333
Mismatched bits count: 8
Matching bases bits count: 15
This is the 0th key: None
This is after amplification: Non-secure key
----------------------------------------------------------------------------------------------------------------
Round 2
Instruction(name='circuit-180', num_qubits=29, num_clbits=29, params=[])
Mismatched bits count: 8
Matching bases bits count: 17
QBER: 0.47058823529411764
Mismatched bits count: 8
Matching bases bits count: 17
This is the 1th key: None
This is after amplification: Non-secure key
----------------------------------------------------------------------------------------------------------------
Round 3
Instruction(name='circuit-189', num_qubits=29, num_clbits=29, params=[])
Mismatched bits count: 8
Matching bases bits count: 15
QBER: 0.5333333333333333
Mismatched bits count: 8
Matching bases 

In [ ]:
class bb84_tester(unittest.TestCase):

  def test_random_bits_string(self):
    # Check length of a string
    self.assertTrue(len(random_bits_string(10))==10)
    string = random_bits_string(10)
    for s in string:
      # Check a string
      self.assertTrue(s=="0" or s=="1")

  def test_random_basis(self):
    # Check length of basis
    self.assertTrue(len(random_basis(10))==10)
    basis = random_basis(10)
    for b in basis:
      # Check a basis
      self.assertTrue(b=="X" or b=="Z")

  def test_encode_qubits(self):
    bits = "1010"
    circuit = encode_qubits(bits)
    # Check if the correct gates are applied based on the bits
    self.assertEqual(circuit.data[0][0].name, 'x')  # First bit is '1', so X gate
    self.assertEqual(circuit.data[1][0].name, 'x')  # Third bit is '1', so X gate
    self.assertEqual(len(circuit.data), 2) # Only two X gates should be applied

  def test_measure_qubits(self):
    # Create a simple circuit to test measurement
    qc = QuantumCircuit(2)
    bob_basis = "XZ"
    measured_circuit = measure_qubits(qc, bob_basis)

    # Check if H gate is applied for 'X' basis
    self.assertEqual(measured_circuit.data[0][0].name, 'h')

    # Check if measure_all is applied
    self.assertEqual(measured_circuit.data[2][0].name, 'measure')


  def test_run_simulator(self):
    nempty = []
    cir = encode_qubits("1010")
    circuit = measure_qubits(cir,"XZXZ")
    result = run_simulator(circuit,nempty)
    self.assertTrue(nempty != [])

  def test_run_simulator_with_noise(self):
    nempty = []
    cir = encode_qubits("1010")
    circuit = measure_qubits(cir,"XZXZ")
    result = run_simulator(circuit,nempty)
    self.assertTrue(nempty != [])

  def test_sift_keys(self):
    alice_basis = "ZZXX"
    bob_basis = "ZXZX"
    alice_bits = "0110"
    shift_key = sift_keys(alice_basis, bob_basis, alice_bits)
    self.assertEqual(shift_key, "00", msg="key was created in wrong way")

  def test_qber(self):
    bob_data = []
    cir = encode_qubits("1010")
    circuit = measure_qubits(cir,"XZXZ")
    run_simulator(circuit, bob_data)
    q = qber("1010",bob_data,"ZXZX","XZXZ")
    qq = qber("1010",bob_data, "XZXZ", "XZXZ")
    self.assertEqual(q, 0.0)
    self.assertTrue(qq == 0.0 or qq == 0.25 or qq == 0.5 or qq ==0.75 or qq == 1)

  def test_amplification(self):
    key = bb84()

    if key != 'None':
      h = amplification(key)
      self.assertTrue(len(h)==64)

In [20]:
if __name__ == '__main__':
  key = bb84()
  h = amplification(key) if key != 'None' else 'Non-secure key'
  print(f"This is the {i}th key: " + str(key))
  print("This is after amplification: " + h)

Instruction(name='circuit-261', num_qubits=29, num_clbits=29, params=[])
Mismatched bits count: 5
Matching bases bits count: 10
QBER: 0.5
Mismatched bits count: 5
Matching bases bits count: 10
This is the 9th key: None
This is after amplification: Non-secure key


In [21]:
if __name__ == '__main__':
  print("-------------------------------------------------------TEST-------------------------------------------------------------------")
  unittest.main(argv=[""], exit=False)

-------------------------------------------------------TEST-------------------------------------------------------------------
Instruction(name='circuit-270', num_qubits=29, num_clbits=29, params=[])


./tmp/ipykernel_8900/558058590.py:20: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  self.assertEqual(circuit.data[0][0].name, 'x')  # First bit is '1', so X gate
/tmp/ipykernel_8900/558058590.py:21: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  self.assertEqual(circuit.data[1][0].name, 'x')  # Third bit is '1', so X gate
./tmp/ipykernel_8900/558058590.py:32: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  self.assertEqual(measured_circuit.data[0][0].name, 'h')
/tmp/ipykernel_8900/5

Mismatched bits count: 13
Matching bases bits count: 19
QBER: 0.6842105263157895
Mismatched bits count: 13
Matching bases bits count: 19
Instruction(name='circuit-279', num_qubits=4, num_clbits=4, params=[])
Instruction(name='circuit-283', num_qubits=4, num_clbits=4, params=[])


...

Mismatched bits count: 4
Matching bases bits count: 4
Instruction(name='circuit-287', num_qubits=4, num_clbits=4, params=[])


.

Instruction(name='circuit-291', num_qubits=4, num_clbits=4, params=[])


..
----------------------------------------------------------------------
Ran 9 tests in 1.834s

OK
